In [ ]:
import time
import requests
import re
import selenium.common.exceptions
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

# Get stories from creepypasta.com
def creepy_url_get():
    option = webdriver.FirefoxOptions()
    option.add_argument("-incognito")
    option.add_argument("--headless")

    print("Getting text urls from creepypasta.com")
    browser = webdriver.Firefox(executable_path="webdriver/geckodriver.exe", options=option)
    browser.set_window_size(1920, 1080)

    # Page load time
    timeout = 20
    
    # Browser setup
    browser.get(f'https://www.creepypasta.com/archive/top-ranked/?_orderby=_gdrts_stars-rating_rating%2Cdesc')
    WebDriverWait(browser, timeout).until(ec.visibility_of_element_located((By.XPATH, f'//li/a[@title="Current page is 1"]')))
    
    urls = []
    
    with open('urls.txt', 'a') as f:
        for i in range(1, 151): # 151
            if i % 10 == 0:
                print(f'Page: {i}. {len(urls)} story links found.')
            if i == 5:
                print(f"Got url:'{url}'")
            browser.get(f'https://www.creepypasta.com/archive/top-ranked/?_orderby=_gdrts_stars-rating_rating%2Cdesc&_page={i}')
            WebDriverWait(browser, timeout).until(ec.visibility_of_element_located((By.XPATH, f'//li/a[@title="Current page is {i}"]')))
            try:
            
                # Hides iframe advertisements to improve speed and improve Selenium's functionality
                iframes = browser.find_elements_by_tag_name("iframe")
                if len(iframes) > 0:
                    browser.execute_script(
                        """
                        var elems = document.getElementsByTagName("iframe");
                            for(var i = 0, max = elems.length; i < max; i++){
                            elems[i].hidden=true;
                        }
                        var ads = document.getElementsByClassName("ad-tag");
                        for(var i =0, max = ads.length; i < max; i++){
                            ads[i].hidden=true;
                        }
                        var ads = document.getElementsByClassName("ad-sticky");
                        for(var i =0, max = ads.length; i < max; i++){
                            ads[i].hidden=true;
                        }
                        var ads = document.getElementsByClassName("ad-sticky-desktop-anchor");
                        for(var i =0, max = ads.length; i < max; i++){
                            ads[i].hidden=true;
                        }
                        """)
                texts = []
            
                # Finds all urls that point to stories based on class identifier
                texts = browser.find_elements_by_xpath("//a[@class='_self cvplbd']")
                for text in texts:
                    try:
                        url = text.get_attribute('href')
                        f.write(url+'\n')
                        urls.append(url)
                    except Exception as e:
                        print(f"Error 1: {e}")
                        break
            except Exception as e:
                print(f"Error 2: {e}")
                break
            browser.find_element_by_xpath(f'//li/a[@title="Go to page {i+1}"]').click()
        time.sleep(1)
    browser.quit()
    f.close()
    print(f'{len(urls)} urls saved.')
    
    
def creepy_requests(urls):
    print("Downloading texts from creepypasta.com")
    creepy_stories = []
    j = 1
    for url in urls:
        if j % 250 == 0:
            print(f'{j}/{len(urls)}')
        r = requests.get(url)
        if r.ok:                
            # bs4 parsing
            soup = bs(r.content, 'html.parser')
                        
            text = []
            paras = soup.find_all('p', href_= None, id_= None)
            for sente in paras:
                sente = sente.get_text()
                sente = sente.replace(u'\xa0', u' ')
                breaks = ["Author's Note: ", "Advertisements", "-Charles ", "STORY BY ", "Credit To", "Credit :", "Credit: ", "CREDIT:", "CREDIT :", "Written ", "WRITTEN ", "Thanks Again, ", "The author of this story wrote it for free."]
                if not any(word in sente for word in breaks):
                    text.append(sente.strip())
                else:
                    break
                story = ' '.join(text)
            creepy_stories.append(story.strip())
            time.sleep(0.3)
        j += 1
    df = pd.DataFrame(creepy_stories, columns=['stories'])
    df.to_csv("stories.csv")
    print(f'{len(creepy_stories)} stories downloaded.')

    
# Get urls
creepy_url_get()

# Get texts and save to csv
with open('urls.txt', 'r') as f:
    urls = f.read().splitlines()
    creepy_requests(urls)